**Scraping information about russian churches**



**The purpose of the work is to collect information about Russian churches, monasteries and chapels for the White Iris charity organization, which is engaged in the restoration of abandoned churches**

Data source: http://www.temples.ru

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from requests_html import AsyncHTMLSession
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.keys import Keys
import math
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import numpy as np

**1 Collection of links by region**

In [2]:
regions = []
req = requests.get('http://www.temples.ru/tree.php')
req.encoding = req.apparent_encoding
soup = BeautifulSoup(req.text)
table = soup.find_all('table')[1]
for region in table.find_all('a', class_ = 'Locate'):
    regions.append('http://www.temples.ru/branch.php?Branch' + region['href'].replace('tree.php?', '') + '&mode=0')

**2 Collection of links to cards for each object belonging to a particular region**

In [3]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [C:\Users\Utilisateur\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2684\4052773410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
cards = []
for region in regions:
    driver.get(region)
    element_present = EC.presence_of_element_located((By.XPATH, "//table[@class='main_data_table']"))
    WebDriverWait(driver, 40).until(element_present)
    html = driver.find_element(By.XPATH, "//table[@class='main_data_table']").get_attribute('innerHTML')
    soup = BeautifulSoup(html)
    for link in soup.find_all('a', class_ = 'isp_prav'):
        cards.append('http://www.temples.ru/' + link['href'])

**3 Collection of object reference information**

In [5]:
len(cards)

29484

Scraping of 1000 links takes about 20 minutes.

Block try/except is added because some links are empty (1 for the first 20 000 links). 

Tables are not the same by its content that's why information was scraped in the text form.

Below is an example to get data about the first thousand objects

In [6]:
info = []
for i in range(1000):
    try:
        r = requests.get(cards[i])
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text)
        table = soup.find('table', bgcolor = "#999999").text
        info.append(table)
    except:
        info.append(np.nan)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

**4 Dataframe creation**

In [7]:
df = pd.DataFrame()
df['info'] = info
df['link'] = cards[:1000]

In [8]:
df['info'].isna().sum()
df = df.dropna()

**4.1 Object name**

In [9]:
def get_name(cell):
    if 'Обиходные названия' in cell:
        return cell.split('Название')[1].split('Обиходные названия')[0]
    else:
        return cell.split('Название')[1].split('Тип постройки')[0]
df['name'] =  df['info'].apply(get_name)

**4.2 Object type**

In [10]:
def get_type(cell):
    if 'Дата основания' in cell and 'Тип постройки' in cell:
        return cell.split('Тип постройки')[1].split('Дата основания')[0]
    elif 'Дата постройки последнего здания' in cell and 'Тип постройки' in cell:
        return cell.split('Тип постройки')[1].split('Дата постройки последнего здания')[0]
    elif 'Архитектор' in cell and 'Тип постройки' in cell:
        return cell.split('Тип постройки')[1].split('Архитектор')[0]
    elif 'Тип постройки' not in cell:
        return np.nan
    else: return cell
df['type_building'] = df['info'].apply(get_type)

**4.3 Object creation date**

In [11]:
def get_date(cell):
    if 'Дата основания' in cell and 'Дата постройки' in cell:
        return cell.split('Дата основания')[1].split('Дата постройки')[0]
    elif 'Дата основания' in cell and 'Архитектор' in cell:
        return cell.split('Дата основания')[1].split('Архитектор')[0]
    elif 'Архитектор' in cell and 'Дата основания' in cell:
        return cell.split('Дата основания')[1].split('Архитектор')[0]
    elif 'Дата основания' not in cell:
        return np.nan
    else: return cell
df['date_creation'] = df['info'].apply(get_date)

**4.4 Object last construction date**

In [12]:
def get_date_last_building(cell):
    if 'Дата постройки последнего здания' in cell and 'Архитектор' in cell:
        return cell.split('Дата постройки последнего здания')[1].split('Архитектор')[0]
    elif 'Дата постройки последнего здания' in cell and 'Историческое исповедание' in cell:
        return cell.split('Дата постройки последнего здания')[1].split('Историческое исповедание')[0]
    elif 'Дата постройки последнего здания' not in cell:
        return 'unknown'
    else: return cell
df['date_last_building_construction'] = df['info'].apply(get_date_last_building).apply(lambda x: x.replace('(см. постройки той же эпохи)', ''))

**4.5 Object status**

In [13]:
def get_status(cell):
    if 'Статус' in cell and 'Адрес на' in cell:
        return cell.split('Статус')[1].split('Адрес на')[0]
    elif 'Статус' in cell and 'Современный адрес' in cell:
        return cell.split('Статус')[1].split('Современный адрес')[0]
    elif 'Статус' in cell and 'Краткое описание' in cell:
        return cell.split('Статус')[1].split('Краткое описание')[0]
    elif 'Статус' in cell and 'Престол' in cell:
        return cell.split('Статус')[1].split('Престол')[0]
    elif 'Статус' not in cell:
        return np.nan
    else: return cell
df['status'] = df['info'].apply(get_status)

**4.6 Object address**

In [14]:
def get_address(cell):
    if 'Современный адрес' in cell and 'Краткое описание' in cell:
        return cell.split('Современный адрес')[1].split('Краткое описание')[0]
    elif 'Современный адрес' in cell and 'Престолы' in cell:
        return cell.split('Современный адрес')[1].split('Престолы')[0]
    elif 'Современный адрес' in cell and 'Посвящение' in cell:
        return cell.split('Современный адрес')[1].split('Посвящение')[0]
    elif 'Современный адрес' in cell and 'Изображение' in cell:
        return cell.split('Современный адрес')[1].split('Изображение')[0]
    elif 'Современный адрес' not in cell:
        return 'unknown'
    else: return cell
df['adress'] = df['info'].apply(get_address).apply(lambda x: x.split('Webhttp')[0]).apply(lambda x: x.split('Престол')[0])

**4.7 Object description**

In [15]:
def get_description(cell):
    if 'Краткое описание' in cell and 'Престол' in cell:
        return cell.split('Краткое описание')[1].split('Престол')[0]
    elif 'Краткое описание' in cell and 'Посвящение' in cell:
        return cell.split('Краткое описание')[1].split('Посвящение')[0]
    elif 'Краткое описание' in cell and 'Примечания' in cell:
        return cell.split('Краткое описание')[1].split('Примечания')[0]
    elif 'Краткое описание' in cell and 'Изображение' in cell:
        return cell.split('Краткое описание')[1].split('Изображение')[0]
    elif 'Краткое описание' not in cell:
        return 'unknown'
    else: return cell
df['description'] = df['info'].apply(get_description)

**4.8 Object coordinates**

In [16]:
def get_coordinates(cell):
    if 'Местоположениекоординаты:' in cell and 'Файл' in cell:
        return cell.split('Местоположениекоординаты:')[1].split('Файл')[0]
    elif 'Местоположениеориентировочные координаты:' in cell  and 'Файл' in cell:
        return cell.split('Местоположениеориентировочные координаты:')[1].split('Файл')[0]
    elif 'Местоположениекоординаты:' not in cell and 'Местоположениеориентировочные координаты:' not in cell:
        return np.nan
    else: return cell
df['coordinates'] = df['info'].apply(get_coordinates)

**4.9 If some help is needed**

In [17]:
list_words = ['пустует', 'требует ремонта', 'заброшен', 'в аварийном состоянии']
def get_needed_help(table):
    for word in list_words:
        if word in table['description'].lower() and table['status'] != "действ.":
            return True
    return False
df['needs_help'] = df.apply(get_needed_help, axis = 1)

In [18]:
df

,info,link,name,type_building,date_creation,date_last_building_construction,status,adress,description,coordinates,needs_help
0,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=9265,Церковь Димитрия Солунского в Алейске,церковь,1983,кон. 1980-х,действ.,"Алтайский край, г. Алейск, ул. Маслозаводская, 39",Старый храм в Алейске закрыт и разрушен в 1960...,52.484150°N 82.801821°E,False
1,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=9264,Церковь Иконы Божией Матери Всех Скорбящих Рад...,церковь,1996,1996-1997,действ.,"Алтайский край, г. Алейск, ул. Комсомольская, 130",Церковь в центральной части города. Устроена в...,52.495021°N 82.771120°E,False
2,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=21445,Церковь Покрова Пресвятой Богородицы в Ае,церковь,NaN,нач. 2010-х,действ.,"Алтайский край, Алтайский р-н, с. Ая, ул. Побе...","Кирпичная церковь, построенная в нач. 2010-х. ...",51.945135°N 85.809354°E,False
3,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=21444,Церковь Покрова Пресвятой Богородицы в Айском ...,церковь,1871,1871-1875,не сохр.,"Алтайский край, Алтайский р-н, с. Ая","Деревянная церковь, обшитая тесом, построенная...",NaN,False
4,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=23184,Церковь Вознесения Господня в Баевском,церковь,не позже сер. XIX в.,1899,не сохр.,"Алтайский кр., Баевский р-н, с. Баево","Деревянная однопрестольная церковь, построена ...",NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
995,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЧасовня...,http://www.temples.ru//card.php?ID=18476,Часовня в Чешьюге,часовня,NaN,нач. ХХ в.,сохр.,"Архангельская обл., Онежский р-н, д. Чешьюга",Деревянная восьмигранная часовня каркасной кон...,63.375115°N 39.108995°E,False
996,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЧасовня...,http://www.temples.ru//card.php?ID=18475,Часовня Варвары Великомученицы в Чешьюге,часовня,NaN,XVIII в. (?),сохр.,"Архангельская обл., Онежский р-н, д. Чешьюга","Деревянная клетская часовня, обшитая тесом, в ...",NaN,False
997,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=16154,Церковь Богоявления Господня в ЧикинскойСинони...,церковь,не позже XVI в.,1904,сохр.,"Архангельская обл., Пинежский р-н, д. Чикинская","Деревянная церковь, обшитая тесом, построенная...",64.489886°N 44.128100°E,True
998,У Ч Е Т Н А Я К А Р Т О Ч К АНазваниеЦерковь...,http://www.temples.ru//card.php?ID=16153,Церковь Покрова Пресвятой Богородицы в Чикинск...,церковь,не позже XVI в.,1874-1889,сохр.,"Архангельская обл., Пинежский р-н, д. Чикинская","Деревянная церковь, обшитая тесом, построенная...",64.489516°N 44.128475°E,True


All information for 30000 objects - https://docs.google.com/spreadsheets/d/12VFkHvx8Qdm_WzR_7XRH4Np_LI7712rcooEuaDtovH0/edit?usp=sharing